In [ ]:
%pip install torch clickhouse_connect scipy transformers accelerate

In [ ]:
import clickhouse_connect, torch
from scipy.spatial import distance
from transformers import AutoModel, AutoTokenizer, Conversation, pipeline

In [ ]:
def search_results(connection, table_name: str, vector: list[float], limit: int = 5):
    res = []
    with connection.query(f"SELECT * FROM {table_name}").rows_stream as stream:
        for item in stream:
            name, url, date, num, text, emb = item

            dist = distance.cosine(vector, emb)
            res.append(
                {
                    "name": name,
                    "url": url,
                    "date": date,
                    "num": num,
                    "text": text,
                    "dist": dist,
                }
            )
    res.sort(key=lambda x: x["dist"])
    return res[:limit]


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def txt2embeddings(text: str, tokenizer, model, device="cpu"):
    encoded_input = tokenizer(
        [text],
        padding=True,
        return_tensors="pt",
        truncation=True,
        max_length=512,
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    with torch.no_grad():
        model_output = model(**encoded_input)

    return mean_pooling(model_output, encoded_input["attention_mask"])[0]


def load_models(model):
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModel.from_pretrained(model)
    return tokenizer, model


def load_chatbot(model):
    chatbot = pipeline(
        model=model,
        trust_remote_code=True,
        torch_dtype="auto",
        device_map="cuda",
        task="conversational",
    )
    return chatbot

In [1]:
HOST = "716c-81-5-106-50.ngrok-free.app"
PORT = "80"
TABLE_NAME = "SbertEmb"
MODEL_EMB_NAME = "ai-forever/sbert_large_nlu_ru"
MODEL_CHAT_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
system_prompt = """
INSTRUCT:
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.

If you receive a question that is harmful, unethical, or inappropriate, end the dialogue immediately and do not provide a response. 

If you make a mistake, apologize and correct your answer.

Generate a response based solely on the provided document.

Answer the following question language based only on the CONTEXT provided.

If you are unsure about your answer or the information in the document, suggest contacting support:
Номер телефона: 8 800 300-30-00
Почтовый адрес для письменных обращений: 107016, Москва, ул. Неглинная, д. 12, к. В, Банк России

Отвечай только на русском языке.
"""

In [ ]:
chatbot = load_chatbot(MODEL_CHAT_NAME)
tokenizer, model = load_models(MODEL_EMB_NAME)

In [ ]:
Conversation = ()

In [ ]:
request = "Как производится антимонопольная политика в России?" #String

In [ ]:
chatbot(conversation, max_new_tokens=512, pad_token_id=tokenizer.eos_token_id, temperature=0.7, top_k=100, top_p=0.95, repetition_penalty=2.0, do_sample=True)

In [ ]:
client = clickhouse_connect.get_client(
    host=HOST, port=PORT
    )
print("Ping:", client.ping())

In [ ]:
tokenizer, model = load_models(MODEL_EMB_NAME)